# Deploy Sports Agent to Production

## Business Objective

This notebook demonstrates deploying the sports agent to Amazon Bedrock AgentCore Runtime for production use. This enables:

**Benefits:**
- **Scalability** - Automatic scaling based on demand
- **Managed Infrastructure** - No server management required
- **High Availability** - Built-in redundancy and failover
- **Observability** - CloudWatch metrics and logs
- **Security** - JWT authentication and IAM integration
- **Streaming** - Real-time response streaming

### Architecture

```
Client → AgentCore Runtime (Container) → AgentCore Gateway → Lambda → AWS Services
           ├─ JWT Auth                      ├─ JWT Auth
           ├─ Auto-scaling                  ├─ Knowledge Base
           └─ CloudWatch Logs               └─ DynamoDB
```

**Components:**
1. **AgentCore Runtime** - Managed container service for agent execution
2. **Docker Container** - Packages agent code and dependencies
3. **ECR Repository** - Stores container images
4. **SSM Parameter Store** - Stores gateway URL configuration
5. **Cognito** - Provides JWT tokens for authentication

### Prerequisites

Complete Lab 2 and have:
- AgentCore Gateway with Lambda tools
- Cognito User Pool with OAuth client
- Gateway URL and authentication credentials

### Resources

- [AgentCore Runtime Documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-runtime.html)
- [Docker Best Practices](https://docs.docker.com/develop/dev-best-practices/)

In [ ]:
import boto3
import sys
from pathlib import Path
from IPython.display import display, Markdown

# Import display helpers
sys.path.insert(0, str(Path.cwd().parent / 'helper'))
from display_helper import print_success, print_info, print_result

# Get region
boto_session = boto3.session.Session()
region = boto_session.region_name

# Restore variables from Lab 2
%store -r lab_gateway_id
%store -r lab_gateway_url
%store -r lab_client_id
%store -r lab_client_secret
%store -r lab_user_pool_id
%store -r lab_scope_string
%store -r bda_result
%store -r lab_cognito_discovery_url

print_info(f"Region: {region}")
print_info(f"User Pool ID: {lab_user_pool_id}")
print_info(f"Discovery URL: {lab_cognito_discovery_url}")
print_info(f"Gateway URL: {lab_gateway_url}")

## Store Gateway Configuration

Store the gateway URL in AWS Systems Manager Parameter Store so the runtime can access it.

**Why SSM Parameter Store?**
- The AgentCore Runtime needs the gateway URL to connect to tools
- SSM provides secure, centralized configuration management
- The runtime's IAM role will have permission to read this parameter

In [ ]:
# Store gateway URL in SSM Parameter Store
# The runtime will retrieve this at startup to connect to the gateway
ssm_client = boto3.client('ssm')

ssm_client.put_parameter(
    Name='/sports-agent/lab_gateway_url',
    Value=lab_gateway_url,
    Type='String',
    Overwrite=True
)

print_success("Gateway URL stored in SSM Parameter Store")
print_info(f"Parameter: /sports-agent/lab_gateway_url")
print_info(f"Value: {lab_gateway_url}")

In [ ]:
%%writefile ./script/sports_agent.py
import boto3
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from strands.models import BedrockModel

# Initialize AWS clients
ssm_client = boto3.client('ssm')

# Model configuration
MODEL_ID = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'


def get_ssm_parameter(parameter_name: str) -> str:
    """Get parameter value from AWS Systems Manager Parameter Store."""
    try:
        response = ssm_client.get_parameter(
            Name=parameter_name,
            WithDecryption=True
        )
        return response['Parameter']['Value']
    except Exception as e:
        print(f"Error getting SSM parameter {parameter_name}: {e}")
        return None

# System prompt for the sports video analysis agent
SYSTEM_PROMPT = """You are a sports video analysis assistant that answers user queries about the provided sports video.
You have video metadata, as well as additional tools to get more information from match reports and player table.
Base on the information you obtained from metadata and tools, generate a clear and accurate answer to the user query. 
DO NOT answer queries beyond the game you are reviewing.
"""


# Initialize the Bedrock model
model = BedrockModel(model_id=MODEL_ID, temperature=0.3)

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()


@app.entrypoint
async def invoke(payload, context=None):
    """AgentCore Runtime entrypoint function"""
    user_input = payload.get("prompt", "")
    
    # Access request headers - handle None case
    request_headers = context.request_headers or {}
    
    # Get Client JWT token
    auth_header = request_headers.get('Authorization', '')
    print(f"Authorization header present: {bool(auth_header)}")
    
    # Get Gateway URL from SSM Parameter Store
    gateway_url = get_ssm_parameter("/sports-agent/lab_gateway_url")
    
    if not gateway_url:
        return "Error: Gateway URL not found in SSM Parameter Store (/sports-agent/lab_gateway_url)"
    
    print(f"Using Gateway URL: {gateway_url}")
    
    # Create MCP client and agent within context manager if JWT token available
    if gateway_url and auth_header:
        try:
            mcp_client = MCPClient(lambda: streamablehttp_client(
                url=gateway_url,
                headers={"Authorization": auth_header}
            ))
            
            with mcp_client:
                # Get tools from MCP gateway
                tools = mcp_client.list_tools_sync()
                print(f"Loaded {len(tools)} tools from gateway")
                
                # Create the agent with gateway tools
                agent = Agent(
                    model=model,
                    tools=tools,
                    system_prompt=SYSTEM_PROMPT,
                )
                
                # Invoke the agent
                response = agent(user_input)
                
                # Extract response text
                if isinstance(response, dict):
                    return response.get('output', str(response))
                elif hasattr(response, 'message'):
                    return response.message["content"][0]["text"]
                else:
                    return str(response)
                    
        except Exception as e:
            print(f"MCP client error: {str(e)}")
            return f"Error: {str(e)}"
    else:
        error_msg = []
        if not gateway_url:
            error_msg.append("Missing gateway URL")
        if not auth_header:
            error_msg.append("Missing authorization header")
        return f"Error: {', '.join(error_msg)}"


if __name__ == "__main__":
    app.run()


## Understanding the Agent Code

The agent code above uses `BedrockAgentCoreApp` to create a production-ready agent runtime.

**Key Components:**

1. **BedrockAgentCoreApp** - Framework for AgentCore Runtime deployment
   - Creates HTTP server on port 8080
   - Implements `/invocations` endpoint for requests
   - Implements `/ping` endpoint for health checks
   - Handles content types and error responses

2. **@app.entrypoint Decorator** - Marks the main handler function
   - Receives `payload` with user input
   - Receives `context` with request metadata (headers, session info)
   - Returns response string or dict

3. **Request Flow:**
   ```
   1. Runtime receives request with JWT token in Authorization header
   2. Extracts prompt from payload
   3. Retrieves gateway URL from SSM Parameter Store
   4. Creates MCP client with JWT token
   5. Lists available tools from gateway
   6. Creates agent with tools and model
   7. Invokes agent with prompt
   8. Returns response
   ```

4. **Configuration Retrieval:**
   - Gateway URL stored in SSM Parameter Store
   - Retrieved at runtime (not hardcoded)
   - Allows updating gateway without redeploying agent

5. **Authentication:**
   - JWT token passed in Authorization header
   - Token forwarded to gateway for tool invocation
   - End-to-end authentication from client to backend services

**Reference:** [AgentCore Runtime SDK](https://pypi.org/project/bedrock-agentcore-runtime/)

## Deploy to AgentCore Runtime

Deploy the agent using the AgentCore Starter Toolkit.

**What is AgentCore Runtime?**
- Managed container service for running agents in production
- Automatically scales based on request volume
- Provides built-in monitoring and logging
- Supports JWT authentication and IAM integration

**Deployment Process:**
1. **Configure** - Define entrypoint, execution role, and authentication
2. **Build** - Create Docker container with agent code
3. **Push** - Upload container to Amazon ECR
4. **Deploy** - Create AgentCore Runtime with container
5. **Monitor** - Check deployment status

**Reference:** [AgentCore Runtime Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-runtime.html)

### Get Access Token

Request a JWT access token from Cognito for testing the deployed runtime.

**Token Details:**
- **Validity:** 1 hour (default, configurable in Cognito)
- **Type:** Bearer token for Authorization header
- **Scopes:** gateway:read and gateway:write
- **Renewal:** Request new token when expired

**Note:** Tokens expire after 1 hour. If you get authentication errors, request a new token using the cell below.

In [ ]:
sys.path.insert(0, '..')

from helper.cognito_helper import CognitoHelper

# Initialize Cognito helper
cognito_helper = CognitoHelper()

from helper.agentcore_helper import AgentCoreHelper

# Initialize AgentCore helper
agentcore_helper = AgentCoreHelper()

In [ ]:
# Get access token
token_response = cognito_helper.get_token(
    user_pool_id=lab_user_pool_id,
    client_id=lab_client_id,
    client_secret=lab_client_secret,
    scope_string=lab_scope_string
)

token = token_response["access_token"]
print_info("Token response: \n" + token)


### Configure AgentCore Runtime

Configure the runtime deployment with authentication and execution permissions.

**Configuration Parameters:**

1. **Entrypoint** - `script/sports_agent.py`
   - Python file with `BedrockAgentCoreApp` and `@app.entrypoint`
   - Entry point for request handling

2. **Execution Role** - IAM role for runtime permissions
   - Grants access to SSM Parameter Store
   - Allows CloudWatch Logs writing
   - Enables Bedrock model invocation

3. **Auto-create ECR** - Automatically creates ECR repository
   - Stores Docker container images
   - Manages image versioning

4. **Requirements File** - `script/requirements.txt`
   - Lists Python dependencies
   - Installed during container build

5. **Authorizer Configuration** - JWT authentication setup
   - **allowedClients** - List of Cognito client IDs that can invoke runtime
   - **discoveryUrl** - Cognito OIDC endpoint for token validation
   - Validates JWT tokens before processing requests

6. **Request Header Configuration** - Headers forwarded to agent code
   - **Authorization** - JWT token for gateway authentication
   - **Custom Headers** - Additional headers if needed
   - Agent code accesses via `context.request_headers`

**What Gets Generated:**
- `Dockerfile` - Container build instructions
- `.bedrock_agentcore.yaml` - Runtime configuration
- Build artifacts in `.bedrock_agentcore/` directory

**Reference:** [Runtime Configuration](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-runtime-config.html)

In [ ]:
lab_agent_name = "lab_sports_agent"
lab_agent_execution_role_name = f"{lab_agent_name}_role"
lab_agent_execution_policy_name = f"{lab_agent_name}_policy"

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = agentcore_helper.create_agentcore_runtime_execution_role(
    role_name=lab_agent_execution_role_name,
    policy_name=lab_agent_execution_policy_name
)

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="script/sports_agent.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="script/requirements.txt",
    region=region,
    agent_name=lab_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [lab_client_id],
            "discoveryUrl": lab_cognito_discovery_url,
        }
    },
    # Add custom header allowlist for Authorization and custom headers
    request_header_configuration={
        "requestHeaderAllowlist": [
            "Authorization",  # Required for OAuth propogation
            "X-Amzn-Bedrock-AgentCore-Runtime-Custom-H1",  # Custom header
        ]
    },
)

print_success("Runtime configuration completed")
print_info(f"Agent name: {lab_agent_name}")
print_info(f"Entrypoint: script/sports_agent.py")

**Note:** This step might fail if the agent with the same name already exists. If you want to overwrite the existing Runtime, use this instead:

 launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)


In [ ]:
launch_result = agentcore_runtime.launch(
    auto_update_on_conflict=True
)
print_success("Runtime launched successfully!")
print_info(f"Agent ARN: {launch_result.agent_arn}")
print_info(f"Agent ID: {launch_result.agent_id}")


### Check Deployment Status

Monitor the runtime deployment until it's ready to accept requests.

**Runtime States:**
- `CREATING` - Container is being deployed
- `UPDATING` - Runtime is being updated
- `READY` - Runtime is ready to accept requests
- `CREATE_FAILED` - Deployment failed
- `UPDATE_FAILED` - Update failed

**What's Happening:**
- Container image is being pulled from ECR
- Runtime environment is being provisioned
- Health checks are being performed
- Auto-scaling is being configured


In [ ]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

if status == 'READY':
    print_success(f"Runtime is ready! Status: {status}")
else:
    print(f"Final status: {status}")


## Test the Deployed Runtime

Invoke the deployed runtime with a test query.

**Invocation Process:**

1. **Prepare Payload** - Create request with prompt and video metadata
2. **Include JWT Token** - Pass bearer token for authentication
3. **Create Session** - Use session ID for conversation continuity
4. **Invoke Runtime** - Send request to AgentCore Runtime endpoint
5. **Receive Response** - Get agent's answer

**What Happens:**
```
1. Runtime validates JWT token against Cognito
2. Forwards request to container with Authorization header
3. Agent code retrieves gateway URL from SSM
4. Agent connects to gateway with JWT token
5. Agent lists and invokes tools as needed
6. Agent returns final answer
7. Runtime streams response back to client
```

**Reference:** [Runtime Invocation](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-runtime-invoke.html)

In [ ]:
# Initialize the AgentCore Control client
client = boto3.client("bedrock-agentcore-control")

# Extract runtime ID from the ARN (format: arn:aws:bedrock-agentcore:region:account:runtime/runtime-id)
runtime_id = launch_result.agent_arn.split(":")[-1].split("/")[-1]

print_info(f"Runtime ID: {runtime_id}")

In [ ]:
query = "who are the players in the video?"

PROMPT_TEMPLATE="""
VIDEO Metadata: {video_summaries}

USER QUERY: {query}
"""

prompt = PROMPT_TEMPLATE.replace("{video_summaries}", bda_result["video"]["summary"])
prompt = prompt.replace("{query}", query)

In [ ]:
import uuid
from IPython.display import display, Markdown


# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()

response = agentcore_runtime.invoke(
    {"prompt": prompt},
    bearer_token=token,
    session_id=str(session_id),
)


output = response["response"].replace('\\n', '\n')
print_result("Agent Response", output)

In [ ]:
lab_agent_arn = launch_result.agent_arn
lab_ssm_parameter = "/sports-agent/lab_gateway_url"
%store lab_agent_arn
%store lab_agent_execution_role_name
%store lab_agent_execution_policy_name
%store lab_ssm_parameter


## 🎉 Lab Complete!

You've successfully deployed a production-ready agent to AgentCore Runtime!

**What You Built:**
- **Production Agent** - Containerized agent running on managed infrastructure
- **Auto-scaling** - Automatically handles varying request loads
- **Secure Authentication** - End-to-end JWT authentication
- **Configuration Management** - Gateway URL stored in SSM Parameter Store
- **Observability** - CloudWatch logs and metrics

**Architecture Achieved:**
```
Client → AgentCore Runtime → AgentCore Gateway → Lambda → AWS Services
         (JWT Auth)           (JWT Auth)          ├─ Knowledge Base
         (Auto-scale)         (MCP Protocol)      └─ DynamoDB
```

**Benefits:**
- No server management required
- Automatic scaling and high availability
- Built-in monitoring and logging
- Secure token-based authentication
- Easy updates without downtime

### Resources

- [AgentCore Runtime Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html)
- [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit)